In [55]:
import pandas as pd
import numpy as np
import math
import csv
import matplotlib.pyplot as plt
import sympy as smp
import re
from tqdm import tqdm
from scipy.linalg import svd
import scipy
from scipy.optimize import curve_fit 
import time
import os

In [56]:
from mat4py import loadmat

In [57]:
def avg(volt,curr,delta):
    V_avg,I_avg = [],[]
    for i in range(len(volt)):
        if i <delta:
            V_avg.append(volt[i])
            I_avg.append(curr[i])
        elif i>=delta:
            V_avg.append(np.mean(volt[i-delta:i])) 
            I_avg.append(np.mean(curr[i-delta:i]))
    return V_avg,I_avg

In [4]:
ExcelFiles = r'D:\ExcelFilesLGData'
data_LG = r'E:\Data_practise\LG_HG2_Original_Dataset_McMasterUniversity_Jan_2020'
if not os.path.isdir(ExcelFiles ):
    os.makedirs(ExcelFiles)

In [74]:
temperature =  [0,10,-10,-20]
sheetnamelist = []     # this list is used to name the sheets
for temp in tqdm(temperature):
    excel_file = pd.ExcelWriter(r'D:\ExcelFilesLGData\{}degC.xlsx'.format(temp))
    matfiledir = r'C:\Users\thiru\OneDrive\Desktop\DATA_DEEPLEARNING\Panasonic 18650PF Data\{}degC\Drive cycles'.format(temp)
    fileslist = os.listdir(matfiledir)
    for matfiles in fileslist:
        matfilepath = os.path.join(matfiledir,matfiles)
        matdata = loadmat(matfilepath)['meas']
        voltage = [i[0] for i in matdata['Voltage']]
        current = [i[0] for i in matdata['Current']]
        Ah = [i[0] for i in matdata['Ah']]
        max_cap = np.max(np.abs(Ah))
        soc = list(map(lambda x: ((max_cap-x)/max_cap)*100,np.abs(Ah)))
        CellTemp = [i[0] for i in matdata['Battery_Temp_degC']]
        V_avg,I_avg = avg(voltage,current,399)                                     # data avg window size of 399 
        DictData = {'voltage':[],'current':[],'V_avg':[],'I_avg':[],'soc':[],'Temp':[]}
        DictData['voltage'] = voltage
        DictData['current'] = current
        DictData['soc'] = soc
        DictData['Temp'] = CellTemp
        DictData['V_avg']= V_avg
        DictData['I_avg']= I_avg
        DictDataDf = pd.DataFrame(DictData)
        ExcelFilePath = os.path.join(ExcelFiles,'{}degCExcelFiles.xlsx'.format(temp))
        name = matfiles.split(' ')[-1].split('.')[0].split('_')[1]

        if name in sheetnamelist:
            sheetname = '{}_{}'.format(name,sheetnamelist.count(name))
            sheetnamelist.append(name)
        else:
            sheetname = name
            sheetnamelist.append(sheetname)
        DictDataDf.to_excel(excel_file, sheet_name=sheetname, index=False)
    excel_file.save()
    excel_file.close()  

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [20:07<00:00, 301.77s/it]
